In [1]:
import tensorflow as tf 
import pickle
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Activation, Dropout
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, accuracy_score
from keras.layers.core import Dense
import numpy as np
import matplotlib.pyplot as plt
from physionet_processing import (fetch_h5data, spectrogram, 
                                  special_parameters, transformed_stats)

from physionet_generator import DataGenerator

In [2]:
# X = pickle.load(open("/Users/sanda/Documents/ptb_xl/ptb_xl_v2_pcl/all_X_ptbxl.pcl", "rb"))
# y = pickle.load(open("/Users/sanda/Documents/ptb_xl/ptb_xl_v2_pcl/all_y_ptbxl.pcl", "rb"))

X = pickle.load(open("/Users/macbookpro/Documents/ptb_xl/scp_filter_v2/all_X_ptbxl.pcl", "rb"))
y= pickle.load(open("/Users/macbookpro/Documents/ptb_xl/scp_filter_v2/all_y_ptbxl.pcl", "rb"))

# X = pickle.load(open("/scratch/thurasx/ptb_xl/scp_filter_v2/all_X_ptbxl.pcl", "rb"))
# y= pickle.load(open("/scratch/thurasx/ptb_xl/scp_filter_v2/all_y_ptbxl.pcl", "rb"))


In [6]:
#comvert X to
# Maximum sequence length
max_length = 1000
# Output dimensions
sequence_length = max_length
spectrogram_nperseg = 32 # Spectrogram window
spectrogram_noverlap = 8 # Spectrogram overlap
batch_size = 32

# calculate image dimensions
_, _, Sxx = spectrogram(X, nperseg = spectrogram_nperseg, noverlap = spectrogram_noverlap)


ValueError: axes don't match array